Project to build and train a DCGAN

In [1]:
import tensorflow as tf
from tensorflow import keras
print('tensorflow version: {}'.format(tf.__version__))
print('keras version: {}'.format(keras.__version__))
print('GPU: {}'.format(tf.config.list_physical_devices('GPU')))

tensorflow version: 2.1.0
keras version: 2.2.4-tf
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np

While in this project, we ultimately wish to train the GAN on the [lfw](https://www.tensorflow.org/datasets/catalog/lfw) dataset to make it generate human faces, we will start small and train it on the Fashion-MNIST for practice. Once that works, we will try to build upon our acquired knowledge to train on the lfw dataset. 

## load the Fashion-MNIST dataset

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train, _), (_,_) = fashion_mnist.load_data()

In [4]:
print('shape of X_train: {}'.format(X_train.shape))
print('max pixel value: {}'.format(X_train.max()))
print('min pixel value: {}'.format(X_train.min()))

shape of X_train: (60000, 28, 28)
max pixel value: 255
min pixel value: 0


In [5]:
# scale the pixel value to have max = 1 and min = -1
max_px = X_train.max()
min_px = X_train.min()
A = 2/(max_px - min_px)
B = -1*(max_px + min_px)/(max_px - min_px)
X_train = A*X_train + B
print('new pixel max: {}'.format(X_train.max()))
print('new pixel min: {}'.format(X_train.min()))

new pixel max: 1.0
new pixel min: -1.0


## Build a DCGAN

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Reshape, Flatten
from tensorflow.keras.layers import Conv2D, Conv2DTranspose

### Generator:

In [7]:
# function to build a generator of given specifications
# Assumption: the first layer is a dense layer and 
# the rest are transpose Convolutional layers
# list layers consists of a list of tuples except for 
# the first entry which is a tuple: (num_units, activation) for the dense layer
# the second entry is tuple of two objects: the height and the width of 
# the feature maps obtained from reshaping the output of the first layer
# the rest of the enteries are tuples: (num_units, kernel_size, stride, padding, activation) for
# the transpose convolutional layers
# if batch_normalization == True, a BatchNormalization layer will be included after 
# every hidden layer except the first dense layer
# if batch_normalization == False and drop_prob!=0, then 
# a dropout layer is added after every layer except the dense layer
def build_generator(list_layers, coding_size = 100, batch_normalization = True, drop_prob = 0):
    
    dense_units, dense_act = list_layers.pop(0)
    reshape_x, reshape_y = list_layers.pop(0)
    reshape_channels = int(dense_units/(reshape_x*reshape_y))
    
    generator = Sequential()
    generator.add(Dense(units=dense_units, activation = dense_act, 
                        input_shape=[coding_size]))
    generator.add(Reshape([reshape_x, reshape_y, reshape_channels]))
    
    for layer in list_layers:
        if batch_normalization:
            generator.add(BatchNormalization())
        elif drop_prob!=0:
            generator.add(Dropout(drop_prob))
            
        num_units, kernel_size, stride, padding, activation = layer
        trans_conv = Conv2DTranspose(filters = num_units, kernel_size = kernel_size, 
                                     strides = stride, padding = padding, 
                                     activation = activation)
        generator.add(trans_conv)
        
    
    return generator

In [8]:
l1 = (7*7*128, 'selu')
l2 = (7, 7)
l3 = (64, 5, 2, 'same', 'selu')
l4 = (1, 5, 2, 'same', 'tanh')

list_layers = [l1,l2,l3,l4]
generator = build_generator(list_layers)
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 128)         512       
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 1)         1601      
Total params: 840,705
Trainable params: 840,321
Non-trainable params: 384
________________________________________________

### Discriminator

In [9]:
# function to build a generator with given specifications
# list_layers consists of a list of tuples used to describe each convolutional layer
# tuples: (num_units, kernel_size, stride, padding, activation)
# if batch_normalization is True, then each layer is followed by a BatchNormalization layer
# if batch_normalization is False and drop_prob!=0, then each layer is followed by a dropout layer
def build_discriminator(list_layers, input_shape, batch_normalization = False, drop_prob = 0.3):
    
    discriminator = Sequential()
    filters, kernel_size, stride, padding, activation = list_layers.pop(0) 
    conv2d = Conv2D(filters = filters, kernel_size = kernel_size, 
                    strides = stride, padding = padding, 
                    activation = activation, input_shape = input_shape)
    discriminator.add(conv2d)
    
    for filters, kernel_size, stride, padding, activation  in list_layers:
        if batch_normalization:
            discriminator.add(BatchNormalization())
        elif drop_prob!=0:
            discriminator.add(Dropout(drop_prob))
        
        conv2d = Conv2D(filters = filters, kernel_size = kernel_size, 
                    strides = stride, padding = padding, activation = activation, )
        
        discriminator.add(conv2d)
        
        discriminator.add(Flatten())
        discriminator.add(Dense(1, activation = 'sigmoid'))
        
        return discriminator

In [10]:
l1 = (64, 5, 2, 'same', keras.layers.LeakyReLU(0.2))
l2 = (128, 5, 2, 'same', keras.layers.LeakyReLU(0.2))
list_layers = [l1,l2]
input_shape = [28,28,1]
discriminator = build_discriminator(list_layers, input_shape)
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6273      
Total params: 212,865
Trainable params: 212,865
Non-trainable params: 0
_________________________________________________________________


### combine the generator and the discriminator into a GAN

In [11]:
Gan = Sequential([generator, discriminator])

In [13]:
# compiling the discriminator
loss = 'binary_crossentropy'
optimizer = 'rmsprop'
metrics = ['accuracy']
discriminator.compile(loss = loss, optimizer = optimizer, metrics = metrics )
discriminator.trainable = False
Gan.compile(loss = loss, optimizer = optimizer, metrics = metrics)

## Custom training loop for the GAN

In [22]:
# function to train the gan
def train_gan(gan, training_images, num_epochs = 2, batch_size = 32, coding_size = 100):
    
    generator, discriminator = gan.layers
    
    idxs = tf.range(2*batch_size)
    for epoch in tf.range(num_epochs):
        
        start = time.time()
        for true_imgs in training_images:
            
            # generate a set of random codings
            codings = tf.random.normal(shape = (batch_size, coding_size))
            # convert the codings into images
            gan_imgs = generator(codings)
            y_gan = tf.constant([[0.]*batch_size])
            
            # mix the gan images with the true images
            y_true = tf.constant([[1.]*batch_size])
            all_imgs = tf.concat([true_imgs, gan_imgs], axis = 0)
            all_y = tf.concat([y_true, y_gan], axis = 0)
            
            # shuffle the set of mixed images
            shuffled_idxs = tf.random.shuffle(idxs)
            shuffled_imgs = tf.gather(all_imgs, shuffled_idxs)
            shuffled_y = tf.gather(all_y, shuffled_idxs)
            
            # phase 1: train the discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(shuffled_imgs, shuffled_y)
        
            # phase 2: train the generator on a new set of codings
            new_codings = tf.random.normal(shape = (batch_size, coding_size))
            discriminator.trainable = False
            gan.train_on_batch(codings, 1-y_gan)
        
        end = time.time()
        time_take = end - start
        print('\repoch:{0:}/{1:}, time:{2: .2f}s '.format(epoch+1, num_epochs, time_taken), 
              end = '', flush = True)
        print('=', end = '')

## Create a batched dataset from X_train

In [31]:
# cast the type of X_train to float32
X_train = tf.cast(X_train, tf.float32)
# Add a channel dimension to X_train
X_train = tf.reshape(X_train, [-1,28,28,1])
# create a dataset
buffer_size = 10000
batch_size = 32
training_images = tf.data.Dataset.from_tensor_slices(X_train).shuffle(buffer_size = buffer_size)
training_images = training_images.batch(batch_size).prefetch(3)

## Train the GAN

In [32]:
num_epochs = 2
coding_size = 100
train_gan(Gan, training_images, num_epochs, batch_size, coding_size)

ValueError: in converted code:

    /home/prarit/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_eager.py:305 train_on_batch  *
        outs, total_loss, output_losses, masks = (
    /home/prarit/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_eager.py:253 _process_single_batch
        training=training))
    /home/prarit/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_eager.py:167 _model_loss
        per_sample_losses = loss_fn.call(targets[i], outs[i])
    /home/prarit/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/keras/losses.py:221 call
        return self.fn(y_true, y_pred, **self._fn_kwargs)
    /home/prarit/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/keras/losses.py:994 binary_crossentropy
        K.binary_crossentropy(y_true, y_pred, from_logits=from_logits), axis=